In [ ]:
import google.generativeai as genai
import json
import pandas as pd

genai.configure(api_key="Api")

model = genai.GenerativeModel("gemini-2.0-flash")


class SupportAgent:

    def process_ticket(self, ticket):

        prompt = f"""
You are a customer support AI agent.

Analyze the following ticket and return ONLY valid JSON.

Return format:
{{
"ticket_id": "",
"department": "",
"priority": "",
"sentiment": "",
"entities": {{}},
"suggested_response": "",
"escalation_required": false,
"reasoning": ""
}}

Rules:
- Departments: Technical, Billing, Sales, Product, General
- Priority: Low, Medium, High, Critical
- Sentiment: Frustrated, Neutral, Satisfied
- Escalate if angry + high issue
- Extract error codes, version numbers, file sizes

Ticket:
Ticket ID: {ticket.get("Ticket ID")}
Customer Name: {ticket.get("Customer Name")}
Product: {ticket.get("Product Purchased")}
Ticket Description: {ticket.get("Ticket Description")}
"""

        response = model.generate_content(prompt)

        text = response.text.strip()

        try:
            result = json.loads(text)
        except:
            result = {"error": "Invalid JSON returned", "raw": text}

        return result


agent = SupportAgent()

ticket = {
    "Ticket ID": "T-1001",
    "Customer Name": "John",
    "Product Purchased": "Subscription",
    "Ticket Description": "My payment failed 3 times and now my account is locked! This is urgent! Error PAY-403"
}

output = agent.process_ticket(ticket)

print(json.dumps(output, indent=2))

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash
Please retry in 26.703729554s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quo

In [ ]:
import os
import re
import json
import pandas as pd
from groq import Groq

GROQ_API_KEY = "Api"

client = Groq(api_key=GROQ_API_KEY)

MODEL_NAME = "openai/gpt-oss-120b"

In [10]:
class Tools:

    @staticmethod
    def call_llm(prompt):
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        return response.choices[0].message.content.strip()

    @staticmethod
    def analyze_sentiment(text):
        prompt = f"""
Classify customer sentiment as one of:
Frustrated, Neutral, Satisfied

Text:
{text}

Return only one word.
"""
        return Tools.call_llm(prompt)

    @staticmethod
    def classify_department(text):
        prompt = f"""
Classify ticket into one department:
Technical, Billing, Sales, Product, General

Text:
{text}

Return only department name.
"""
        return Tools.call_llm(prompt)

    @staticmethod
    def assess_priority(text):
        prompt = f"""
Classify urgency as:
Low, Medium, High, Critical

Text:
{text}

Consider urgency words and severity.

Return only priority.
"""
        return Tools.call_llm(prompt)

    @staticmethod
    def extract_entities(text):
        entities = {}

        error = re.search(r'[A-Z]{2,5}-\d{3,5}', text)
        if error:
            entities["error_code"] = error.group()

        version = re.search(r'v\d+\.\d+(\.\d+)?', text)
        if version:
            entities["version"] = version.group()

        size = re.search(r'\d+MB|\d+GB', text)
        if size:
            entities["file_size"] = size.group()

        prompt = f"""
Extract key structured entities from this ticket.
Return valid JSON.

Text:
{text}
"""
        try:
            llm_entities = json.loads(Tools.call_llm(prompt))
            entities.update(llm_entities)
        except:
            pass

        return entities

    @staticmethod
    def search_faq(text):
        kb = {
            "payment": "Please verify your payment details or try another method.",
            "refund": "Refunds are processed within 5-7 business days.",
            "crash": "Please update to the latest version and clear cache.",
            "locked": "Use password reset to regain access.",
            "pricing": "Our sales team will provide enterprise pricing details."
        }

        for key in kb:
            if key in text.lower():
                return kb[key]

        return "Our team will investigate and respond shortly."

    @staticmethod
    def generate_response(ticket, dept, priority, sentiment, faq):
        prompt = f"""
Generate a professional support email.

Customer Name: {ticket.get("Customer Name")}
Department: {dept}
Priority: {priority}
Sentiment: {sentiment}
Suggested Solution: {faq}

Keep it polite and helpful.
"""
        return Tools.call_llm(prompt)

    @staticmethod
    def log_decision(text, dept, priority):
        prompt = f"""
Explain briefly why this ticket was routed to {dept}
and assigned {priority} priority.

Text:
{text}
"""
        return Tools.call_llm(prompt)





In [11]:

class SupportAgent:

    def process_ticket(self, ticket):

        text = ticket.get("Ticket Description", "")

        sentiment = Tools.analyze_sentiment(text)

        entities = Tools.extract_entities(text)

        department = Tools.classify_department(text)

        priority = Tools.assess_priority(text)

        faq_solution = Tools.search_faq(text)

        escalation = False

        if sentiment == "Frustrated" and priority in ["High", "Critical"]:
            escalation = True

        if "cancel" in text.lower() and "refund" in text.lower():
            entities["multi_issue"] = True
            escalation = True

        response = Tools.generate_response(
            ticket,
            department,
            priority,
            sentiment,
            faq_solution
        )

        reasoning = Tools.log_decision(text, department, priority)

        return {
            "ticket_id": ticket.get("Ticket ID"),
            "department": department,
            "priority": priority,
            "sentiment": sentiment,
            "entities": entities,
            "suggested_response": response,
            "escalation_required": escalation,
            "reasoning": reasoning
        }


In [12]:
if __name__ == "__main__":

    agent = SupportAgent()

    data = pd.read_csv(r"C:\Users\MITHUN\Desktop\STUDIES\Drive\Akaike\Problem-B\dataset\customer_support_tickets.csv")

    sample = data.head(3)

    for _, row in sample.iterrows():
        ticket = row.to_dict()
        result = agent.process_ticket(ticket)
        print(json.dumps(result, indent=2))

{
  "ticket_id": 1,
  "department": "Technical",
  "priority": "Medium",
  "sentiment": "Frustrated",
  "entities": {},
  "suggested_response": "**Subject:** Your Technical Support Request \u2013 We\u2019re On It (Priority: Medium)\n\n---\n\nHi Marisa,\n\nThank you for reaching out to the Technical Support team. I\u2019m sorry to hear that you\u2019re experiencing difficulties, and I understand how frustrating this can be.\n\n**What\u2019s happening:**  \n- **Customer:** Marisa O\u2019Brien  \n- **Department:** Technical  \n- **Priority:** Medium  \n- **Current Sentiment:** Frustrated  \n\n**Next steps:**  \nOur team is already investigating the issue you reported. We\u2019ll gather the necessary details, run diagnostics, and work toward a resolution as quickly as possible. You can expect an update from us within the next\u202f24\u202fhours, or sooner if we have a definitive answer.\n\nIn the meantime, if you notice any new information or additional symptoms, please feel free to reply 

In [13]:
import gradio as gr
import json

agent = SupportAgent()

def process_ticket_ui(ticket_id, customer_name, product, description):

    ticket = {
        "Ticket ID": ticket_id,
        "Customer Name": customer_name,
        "Product Purchased": product,
        "Ticket Description": description
    }

    result = agent.process_ticket(ticket)

    return json.dumps(result, indent=2)


demo = gr.Interface(
    fn=process_ticket_ui,
    inputs=[
        gr.Textbox(label="Ticket ID"),
        gr.Textbox(label="Customer Name"),
        gr.Textbox(label="Product Purchased"),
        gr.Textbox(label="Ticket Description", lines=6)
    ],
    outputs=gr.Textbox(label="Agent Output (JSON)", lines=20),
    title="Customer Support Ticket Routing Agent",
    description="Enter a support ticket and see automatic routing, priority, sentiment, and response."
)

demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
